In [1]:
from Supporting_functions import *
from aicsimageio.readers import CziReader
from aicsimageio.writers import OmeTiffWriter
import re
import scipy.ndimage as ndi
from WBNS import WBNS_image
import RedLionfishDeconv as rl


# Helper functions

def custom_sort_images(file_name):
    allparts = file_name.split('(')
    parts = allparts[1].split(')')
    return (parts[0].zfill(4))


def open_image_from_reader(reader, view, color, order="ZYX", out_type=np.uint16):
    
    lazy_t0 = reader.get_image_dask_data(order, V=view, C=color)  # returns 3D ZYX numpy array
    img = lazy_t0.compute()  # returns in-memory 4D numpy array
    img = img.astype(out_type)
    
    return img, reader.physical_pixel_sizes.X, reader.physical_pixel_sizes.Z


def image_preprocessing(img, mask, percentiles, min_v, max_v):
    
    # Image Normalization
    if percentiles[0] > 0 or percentiles[1] < 100:
        low_thres, high_thres0 = getNormalizationThresholds(img, percentiles) # low thres in whole image
        low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles) # high thres in FG
        img = remove_outliers_image(img, low_thres, high_thres)
  
    img, scaleI = image_get_scaling(img, min_v, max_v)
    
    return img, scaleI

def get_IsoImage(img0,pixel_size_X,pixel_size_Z,net_Iso,min_v,max_v, device1, device2, batch_size,skip_planes):
    
    # Isotropic prediction 
    img = upsample_block(img0,pixel_size_X,pixel_size_Z,net_Iso,net_Iso,min_v,max_v, device1, device2, batch_size, skip_planes)              
    # Re-scale intensities
    img = img.astype(np.float32) / scaleI
    img = img.astype(np.uint16)

    return img

def image_postprocessing(img, resolution_px, noise_lvl, sigma):
    # Remove noise and BG
    if resolution_px > 0:
        img = WBNS_image(img, resolution_px, noise_lvl)

    # Smooth
    if sigma > 0:
        img = ndi.gaussian_filter(img, sigma)

    return img

/home/hmorales/miniconda3/envs/selfnet/lib/python3.9/site-packages/pydantic/_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


# Predictions for a folder

## Input paramters

In [2]:
# Input data and paths

# Images to analyze
img_src_path = '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/forAnalysis/231218_sqtsfGFP_singletrack_18deg/'
SqtsfGFPChannelId = 0                         # For SqtsfGFP, starting from 0
NucleiMembranesChannelId = 1                  # For Nuclei+Membranes or just Nuclei, starting from 0
angles = ['000'] 

# Output dir
outdir = '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/forAnalysis/231218_sqtsfGFP_singletrack_18deg_tiffs/'

In [3]:
# Define paths for models: 

# Paths for all models
srcpath = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/'

# Model folders
modelNameNuclei       = 'NucleiMembrane2Nuclei'
modelNameMembranes    = 'NucleiMembrane2Membrane'
modelNameIsoNuclei    = 'Iso_Nuclei'
modelNameIsoMembranes = 'Iso_Membranes'
modelNameIsoSqtsfGFP  = 'Iso_SqtsfGFP'

# Model Ids
modelIdNuclei       = ''
modelIdMembranes    = ''
modelIdIsoNuclei    = 'deblur_net_94_96000.pkl'
modelIdIsoMembranes = ''
modelIdIsoSqtsfGFP  = 'deblur_net_140_7200.pkl'

In [4]:
# CUDA device
device1 = torch.device('cuda:0')
device2 = torch.device('cuda:0')
batch_size = 8

skip_planesSqtsfGFP = 2   # for SqtsfGFP Iso Prediction
skip_planesNuclei   = 4   # for Nuclei Iso Prediction
skip_planesMembrane = 0   # for Membrane Iso Prediction

In [5]:
# Crop Images for calculation
crop_for_calculations = False
thres_crop = 2.5
blur_mask = 2.0

# Input Image Normalization
min_v = 0
max_v = 65535
norm_percentiles =  (20, 99.999) 

# Output Image post-processing
crop_images = True
# BG subtraction
resolution_px = 10 # FWHM of the PSF
noise_lvl = 2
sigma = 0.8             #Smooth results 

In [6]:
# Define Models paths and create output folders. Modify only if necessary
model_path_Nuclei       = srcpath+modelNameNuclei      +'/'+'checkpoint/saved_models/'+modelIdNuclei
model_path_Membranes    = srcpath+modelNameMembranes   +'/'+'checkpoint/saved_models/'+modelIdMembranes
model_path_IsoSqtsfGFP  = srcpath+modelNameIsoSqtsfGFP +'/'+'checkpoint/saved_models/'+modelIdIsoSqtsfGFP
model_path_IsoNuclei    = srcpath+modelNameIsoNuclei   +'/'+'checkpoint/saved_models/'+modelIdIsoNuclei
model_path_IsoMembranes = srcpath+modelNameIsoMembranes+'/'+'checkpoint/saved_models/'+modelIdIsoMembranes

# Create output folder
if not os.path.exists(outdir):
    os.mkdir(outdir)         

## Prepare networks

In [7]:
#Create Networks
net_Nuclei    = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
net_Membranes = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')

net_IsoSqtsfGFP  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
net_IsoNuclei    = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
net_IsoMembranes = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')

# Load Models

if modelIdNuclei:
    net_Nuclei.load_state_dict(torch.load(model_path_Nuclei))
if modelIdMembranes:
    net_Membranes.load_state_dict(torch.load(model_path_Membranes))
if modelIdIsoSqtsfGFP:
    net_IsoSqtsfGFP.load_state_dict(torch.load(model_path_IsoSqtsfGFP))
if modelIdIsoNuclei:
    net_IsoNuclei.load_state_dict(torch.load(model_path_IsoNuclei))
if modelIdIsoMembranes:
    net_IsoMembranes.load_state_dict(torch.load(model_path_IsoMembranes))

initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal


## Perform calculations 

### Get time series maks

In [8]:
# Get all czi images in the folder
image_names = sorted([f for f in os.listdir(img_src_path) if f.endswith('.czi')])
image_names.sort(key=custom_sort_images)

# Get images for masking
N = len(image_names)
mN = int(0.5 * (N-1))

times4masking = [0, mN, N-1]

# get image path 
czi_file_path = os.path.join(img_src_path, image_names[0])   
reader = CziReader(czi_file_path)
raw_img, pixel_size_X, pixel_size_Z = open_image_from_reader(reader, 0, NucleiMembranesChannelId, "ZYX", np.uint16)
 
# Sume up all channels
img = np.zeros_like(raw_img)
del raw_img

for idx in times4masking:
    czi_file_path = os.path.join(img_src_path, image_names[idx])   
    reader = CziReader(czi_file_path)
    print('Loading : ', image_names[idx])
    raw_img, pixel_size_X, pixel_size_Z = open_image_from_reader(reader, 0, NucleiMembranesChannelId, "ZYX", np.uint16)
    img += raw_img
    del raw_img
    
# get actual mask and boundary boxes

print(img.shape)
print(f"Getting tissue mask ...")
start_time = time.time()
        
scale = pixel_size_X / pixel_size_Z
if crop_images == True:
    bounds_min_full, bounds_max_full, mask = get_image_cropping_box(img, blur_mask, scale, thres_crop)                     

img = img[bounds_min_full[0]:bounds_max_full[0], bounds_min_full[1]:bounds_max_full[1], bounds_min_full[2]:bounds_max_full[2]]    
Elapsed_time = time.time() - start_time
print(f"Elapsed Time masking: {Elapsed_time:.4f} seconds") 
print(img.shape)

# create empty output image as template
z,y,x = img.shape
new_z = round(z / scale) 
base_img  = np.zeros((new_z,y,x))    

outName = 'mask.tif'        
custom_save_img(img, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X) 

Loading :  18degr_singleTrack__2023_12_18__15_49_26_471(190).czi
Loading :  18degr_singleTrack__2023_12_18__15_49_26_471(214).czi
Loading :  18degr_singleTrack__2023_12_18__15_49_26_471(239).czi
(508, 1920, 1920)
Getting tissue mask ...
     -threshold_value: 1535.0
Elapsed Time masking: 200.6157 seconds
(290, 1073, 1346)


### Predict images 

In [9]:
# Loop over all images
image_names = sorted([f for f in os.listdir(img_src_path) if f.endswith('.czi')])
image_names.sort(key=custom_sort_images)

for i, image_name in enumerate(image_names):
       
    # get image path 
    czi_file_path = os.path.join(img_src_path, image_name)   
    reader = CziReader(czi_file_path)
    
    # get time Id 
    match = re.search(r'\((\d+)\)', image_name)
    timeId = match.group(1)
    timeId= timeId.zfill(3)

    print(f"** Processing image : {image_name}")
    for view in range(reader.dims.V ):      
        print(f" -Processing view: {view}")
        # Process color by color     
        for color in range(reader.dims.C): 
            print(f"     -Processing color: {color}")   
            
            start_time = time.time()  # Record the start time 
            
            # Open image
            raw_img, pixel_size_X, pixel_size_Z = open_image_from_reader(reader, view, color, "ZYX", np.uint16)
            scale = pixel_size_X / pixel_size_Z

            if crop_images == True:
                raw_img = raw_img[bounds_min_full[0]:bounds_max_full[0], bounds_min_full[1]:bounds_max_full[1], bounds_min_full[2]:bounds_max_full[2]]
                        
            # Prepare image for calculations

            if crop_for_calculations == True:
                bounds_min, bounds_max, mask = get_image_cropping_box(raw_img, blur_mask, scale, thres_crop)                     
                img = raw_img[bounds_min[0]:bounds_max[0], bounds_min[1]:bounds_max[1], bounds_min[2]:bounds_max[2]]
            else:
                img = raw_img

            # Image Normalization
            img, scaleI = image_preprocessing(img, mask, norm_percentiles, min_v, max_v)
            print(f"     -scaling: {scaleI}")  
                           
            if color == SqtsfGFPChannelId:
                 # Isotropic prediction 
                img = get_IsoImage(img,pixel_size_X,pixel_size_Z,net_IsoSqtsfGFP,min_v,max_v, device1, device2, batch_size,skip_planesSqtsfGFP)                   
                if crop_for_calculations == True:   
                    img = insert_predicted_image(base_img,img,bounds_min,bounds_max,scale)
                img = image_postprocessing(img, resolution_px, noise_lvl, sigma)
                # Save Image
                outName = 'spim_TL'+str(timeId)+'_Channel'+str(color)+'_Angle'+angles[view]+'.tif'        
                custom_save_img(img, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X)  
                
            elif color == NucleiMembranesChannelId:
                if modelIdNuclei and modelIdMembranes:
                    print("TO-DO")
                else:
                    # Isotropic prediction 
                    img = get_IsoImage(img,pixel_size_X,pixel_size_Z,net_IsoNuclei,min_v,max_v, device1, device2, batch_size,skip_planesNuclei)                   
                    if crop_for_calculations == True:   
                        img = insert_predicted_image(base_img,img,bounds_min,bounds_max,scale)
                    img = image_postprocessing(img, resolution_px, noise_lvl, sigma)
                    # Save Image
                    outName = 'spim_TL'+str(timeId)+'_Channel'+str(color)+'_Angle'+angles[view]+'.tif'        
                    custom_save_img(img, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X)                    
 
            Elapsed_time = time.time() - start_time
            print(f"     -Elapsed Time: {Elapsed_time:.4f} seconds")

** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(190).czi
 -Processing view: 0
     -Processing color: 0
     -scaling: 439.8322147651007


Predicting : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:16<00:00,  1.10 planes/s]


     -Elapsed Time: 328.0229 seconds
     -Processing color: 1
     -scaling: 1.0304731355252605


Predicting : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:38<00:00,  1.10 planes/s]


     -Elapsed Time: 241.5321 seconds
** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(191).czi
 -Processing view: 0
     -Processing color: 0
     -scaling: 428.3333333333333


...

     -Elapsed Time: 226.5216 seconds
** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(239).czi
 -Processing view: 0
     -Processing color: 0
     -scaling: 213.46905537459284


Predicting : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:16<00:00,  1.10 planes/s]
/home/hmorales/miniconda3/envs/selfnet/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


     -Elapsed Time: 413.0883 seconds
     -Processing color: 1
     -scaling: 1.0032914880587875


Predicting : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:37<00:00,  1.11 planes/s]


     -Elapsed Time: 228.9573 seconds


In [10]:
"""          
            #  Predict nuclei, membranes and make isotropic image
            if color == NucleiMembranesChannelId:
                
                # prepare image for calculations
                if crop_for_calculations == True:
                    img = raw_img[bounds_min[0]:bounds_max[0], bounds_min[1]:bounds_max[1], bounds_min[2]:bounds_max[2]]
                    # get mask for beads           
                    imgBeads =  img * (~mask)
                else:
                    img = raw_img
                    imgBeads =  raw_img * (~mask)

                # Normalize 
                #low_thres, high_thres = getNormalizationThresholds(img, norm_percentiles)
                #img = remove_outliers_image(img, low_thres, high_thres)
                #img, scaleI = image_get_scaling(img, min_v, max_v)

                img, scaleI = image_preprocessing(img, mask, norm_percentiles, min_v, max_v)
                print("scaleI : ", scaleI)
        
                # Enhance Image
                print("Enhance Image : ")
                #img = predict_stack(img,net_EhnNucleiMem, min_v, max_v, device1, batch_size, np.uint16)       
                #img, scaleI1 = image_preprocessing(img, mask, norm_percentiles, min_v, max_v)
               
                #scaleI = scaleI * scaleI1
                #print("scaleI : ", scaleI)
                
                # Predict image
                print("Predict imgNuclei : ")
                imgNuclei = predict_stack(img,net_Nuclei,    min_v, max_v, device1, batch_size, np.uint16)
                print("Predict imgCells : ")
                imgCells  = predict_stack(img,net_Membranes, min_v, max_v, device1, batch_size, np.uint16)
                
                imgNuclei = imgNuclei * mask
                imgCells  = imgCells  * mask
                
                # Isotropic prediction 
                print("Iso imgNuclei : ")
                imgNuclei = upsample_block(imgNuclei,pixel_size_X,pixel_size_Z,net_IsoNuclei,   net_IsoNuclei,   min_v,max_v, device1, device2, batch_size, skip_planes)              
                print("Iso imgCells : ")
                imgCells  = upsample_block(imgCells, pixel_size_X,pixel_size_Z,net_IsoMembranes,net_IsoMembranes,min_v,max_v, device1, device2, batch_size, skip_planes2)              
            
                # Re-scale intensities
                print("Scaling intensities ...")
                imgNuclei  = imgNuclei.astype(np.float32).astype(np.float32) / scaleI
                imgCells   = imgCells.astype(np.float32) / scaleI
                imgNuclei  = imgNuclei.astype(np.uint16)
                imgCells   = imgCells.astype(np.uint16)

               
                # put image back 
                if crop_for_calculations == True:   
                    ''' 
                    # if not isotropic prediction is done
                    base_img = np.zeros_like(raw_img)
                    imgNuclei = insert_predicted_image(base_img,imgNuclei,bounds_min,bounds_max,1.0)
                    imgCells = insert_predicted_image(base_img,imgCells, bounds_min,bounds_max,1.0)
                    imgBeads = insert_predicted_image(raw_img,imgBeads,bounds_min,bounds_max,1.0)
                    '''
                    imgNuclei = insert_predicted_image(base_img,imgNuclei,bounds_min,bounds_max,scale)
                    imgCells  = insert_predicted_image(base_img,imgCells, bounds_min,bounds_max,scale)
                    imgBeads  = insert_predicted_image(raw_img,imgBeads,bounds_min,bounds_max,1.0)              

                    
                # Make images isotropic 
                '''
                # if not isotropic prediction is done
                imgNuclei = reslice(imgNuclei,'xy',pixel_size_X,pixel_size_Z)
                imgCells  = reslice(imgCells,'xy',pixel_size_X,pixel_size_Z)
                '''
                imgBeads  = reslice_bysize(imgBeads,'xy',new_z)    
                    

                if Niter > 10: 
                    # Padding image
                    imgBeads = np.pad(imgBeads, padding, mode='reflect')
                    imgSizeGB = imgBeads.nbytes / (1024 ** 3)
                    print('     -size(GB) : ', imgSizeGB)
                    # GPU deconvolution
                    res_gpu = rl.doRLDeconvolutionFromNpArrays(imgBeads, psf, niter=Niter,resAsUint8=False)
                    # Removing padding
                    imgBeads = res_gpu[padding:-padding, padding:-padding, padding:-padding]
            
                # Normalize output
                #imgNuclei     = image_normalizing(imgNuclei,norm_percentiles_out, min_v, max_v)
                #imgCells      = image_normalizing(imgCells,norm_percentiles_out, min_v, max_v)
                #imgNucleiTemp = image_normalizing(imgNucleiTemp,norm_percentiles_out, min_v, max_v)

                # Remove noise and BG
                if resolution_px > 0:
                    imgNuclei = WBNS_image(imgNuclei, resolution_px, noise_lvl)
                    imgCells = WBNS_image(imgCells, resolution_px, noise_lvl)

                # Smooth
                if sigma > 0:
                    imgNuclei = ndi.gaussian_filter(imgNuclei, sigma)
                    imgCells = ndi.gaussian_filter(imgCells, sigma)
                
                # Save image
                print("Saving ...")
                outName = 'spim_TL'+str(timeId)+'_Channel'+str(color+2)+'_Angle'+angles[view]+'.tif'        
                custom_save_img(imgNuclei, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X)
                outName = 'spim_TL'+str(timeId)+'_Channel'+str(color+3)+'_Angle'+angles[view]+'.tif'        
                custom_save_img(imgCells, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X)
                outName = 'spim_TL'+str(timeId)+'_Channel'+str(color+4)+'_Angle'+angles[view]+'.tif'        
                custom_save_img(imgBeads, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X)
                 

            else:
                
                if crop_for_calculations == True:
                    img = raw_img[bounds_min[0]:bounds_max[0], bounds_min[1]:bounds_max[1], bounds_min[2]:bounds_max[2]]
                else:
                    img = raw_img
                
                img = img * mask
                fusion_stack=reslice(img,'xy',pixel_size_X,pixel_size_Z)

                if Niter > 0:       
                    # Padding image
                    fusion_stack = np.pad(fusion_stack, padding, mode='reflect')
                    imgSizeGB = fusion_stack.nbytes / (1024 ** 3)
                    print('     -size(GB) : ', imgSizeGB)
                    # GPU deconvolution
                    res_gpu = rl.doRLDeconvolutionFromNpArrays(fusion_stack, psf, niter=Niter,resAsUint8=False)
                    # Removing padding
                    fusion_stack = res_gpu[padding:-padding, padding:-padding, padding:-padding]
                                 
                if crop_for_calculations == True:
                    fusion_stack = insert_predicted_image(base_img,fusion_stack,bounds_min,bounds_max,scale)
                    
                outName = 'spim_TL'+str(timeId)+'_Channel'+str(color)+'_Angle'+angles[view]+'.tif'        
                custom_save_img(fusion_stack, outdir, outName, pixel_size_X,pixel_size_X, pixel_size_X)
  
  
            Elapsed_time = time.time() - start_time
            print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {outName}")                  
 """

'          \n            #  Predict nuclei, membranes and make isotropic image\n            if color == NucleiMembranesChannelId:\n                \n                # prepare image for calculations\n                if crop_for_calculations == True:\n                    img = raw_img[bounds_min[0]:bounds_max[0], bounds_min[1]:bounds_max[1], bounds_min[2]:bounds_max[2]]\n                    # get mask for beads           \n                    imgBeads =  img * (~mask)\n                else:\n                    img = raw_img\n                    imgBeads =  raw_img * (~mask)\n\n                # Normalize \n                #low_thres, high_thres = getNormalizationThresholds(img, norm_percentiles)\n                #img = remove_outliers_image(img, low_thres, high_thres)\n                #img, scaleI = image_get_scaling(img, min_v, max_v)\n\n                img, scaleI = image_preprocessing(img, mask, norm_percentiles, min_v, max_v)\n                print("scaleI : ", scaleI)\n        \n  

In [11]:
imgNuclei.shape

NameError: name 'imgNuclei' is not defined

In [ ]:
'''
scale_img = reslice(raw_img,'xy',reader.physical_pixel_sizes.X,reader.physical_pixel_sizes.Z)            
beads_image =  img * (~mask)
beads_image = insert_predicted_image(scale_img,beads_image,bounds_min,bounds_max,1.0)


outName = 'spim_TL'+str(timeId)+'_Channel'+str(color+2)+'_Angle'+angles[view]+'.tif'        
custom_save_img(beads_image, outdir, outName, reader.physical_pixel_sizes.X, reader.physical_pixel_sizes.Y, reader.physical_pixel_sizes.Z)
'''
